In [1]:
from transformers.models.gemma3_with_talker.configuration_gemma3_with_talker import (
    Gemma3WithTalkerConfig,
)
from transformers.models.gemma3_with_talker.modeling_gemma3_with_talker import (
    Gemma3WithTalkerForConditionalGeneration,
)
from transformers.models.gemma3_with_talker.configuration_gemma3_with_talker import *
from transformers import AutoConfig, AutoModel

from loguru import logger
from transformers import AutoConfig

/data1/nowshad/rpTalker/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/args_doc.py
🚨 Config not found for gemma3-with-talker. You c

In [2]:
import os

os.environ["LIBRARY_PATH"] = "/usr/local/cuda/lib64/stubs:$LIBRARY_PATH"

In [4]:
model = Gemma3WithTalkerForConditionalGeneration.from_pretrained(
    "/data1/nowshad/models/gemma_talker_merge_test",
    device_map="cuda:6",
    # config=config,
    torch_dtype="bfloat16",
)

2025-05-27 21:58:49.188 | INFO     | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4149 - Inside from pretrained
2025-05-27 21:58:49.189 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4150 - Config None
2025-05-27 21:58:49.190 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4151 - Model args
2025-05-27 21:58:49.190 | DEBUG    | transformers.models.gemma3_with_talker.modeling_gemma3_with_talker:from_pretrained:4153 - kwargs
You are using a model of type qwen2_5_omni to instantiate a model of type gemma3_with_talker. This is not supported for all configurations of models and can yield errors.
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


ValueError: embed_dim must be divisible by num_heads (got `embed_dim`: 1280 and `num_heads`: 12).